# Extract features from TIMIT corpus

In [16]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from pathlib import Path
import os

subject_block = 'TIMIT_corpus'

# Shared paths
preproc_root = Path('/userdata/dxu/project_np/preproc')
label_dir = preproc_root / subject_block / 'labels'
chunk_wav_dir = preproc_root / subject_block / 'audio_files' / 'clips'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **1. Collect TIMIT wav files**

In [25]:
import glob
import shutil

timit_dir = Path('/userdata/dxu/project_np/tasks/LMV/TIMIT')
wav_pattern = timit_dir / '*_*.wav'
wav_files = glob.glob(str(wav_pattern))

chunk_wav_dir.mkdir(parents=True, exist_ok=True)

for p in wav_files:
    p_new = chunk_wav_dir / Path(p).name
    shutil.copy(p, p_new)


## **2. Extract features from audio**

Most of the following subsections can be excuted on-demand and in parallel.
The exception is the pitch extraction in 4.4, which requires extracted phonetic labels from 4.2.

### **2.1. Estimate articulatory trajectories**

This uses the bootphon articulatory inversion package. For more information check out their github. https://github.com/bootphon/articulatory_inversion

In [26]:
from preproc_func.aai import invert

artics_dir = preproc_root / subject_block / 'artics'

!rm -rf $artics_dir

invert(chunk_wav_dir, artics_dir)


running inversion!
returning artic output dir and prog. file


### **4.2. Extract phonetic labels**

This uses montreal-forced-aligner. Please ask Many for installation instruction if you need them

In [ ]:
from preproc_func.phoneme.pipeline import run_mfa

speech_df = pd.read_csv(label_dir / 'combined_speech_labels.csv')

phone_dir = preproc_root / subject_block / 'phones'

!rm -rf $phone_dir

run_mfa(chunk_wav_dir, speech_df, phone_dir, language='english')


In [ ]:
# note - its important here to see everything was aligned. If it wasnt that usally means a label was off

In [ ]:
from preproc_func.phoneme.pipeline import pool_speakers

pool_speakers(phone_dir)


### **4.3. Get peakRate, peakEnv, env, and f0 using Yulia's peakRate code**

Run a script that does the matlab peakRate extraction. The results will come out at 100 samples/sec.

In [ ]:
from preproc_func.intensity.pipeline import extract_intensity

intens_dir = preproc_root / subject_block / 'intensity'

!rm -rf intens_dir

extract_intensity(chunk_wav_dir, intens_dir)


### **4.4. Extract pitch features**
#### NOTE: THIS STEP REQUIRES THE COMPLETION OF 4.2

In [ ]:
from preproc_func.pitch.straight import run_straight

tg_dir = preproc_root / subject_block / 'phones/results/all'
pitch_dir = preproc_root / subject_block / 'pitch'

!rm -rf $pitch_dir

run_straight(chunk_wav_dir, tg_dir, pitch_dir)


In [ ]:
from preproc_func.pitch.crepe_lg import make_pitch

pitch_dir = preproc_root / subject_block / 'pitch'
!rm -rf $pitch_dir
pitch_dir.mkdir(parents=True, exist_ok=True)

# Run laura G pitch extraction
make_pitch(chunk_wav_dir, pitch_dir)

### **4.5. Extract formants using PRAAT**
#### WARNING: THIS STEP REQUIRES USER INTERVENTION AS THE SCRIPT HAS TO BE RUN LOCALLY
it'd be great if it didnt :sob:

In [ ]:
numbered_wavdir

In [ ]:
# Step 1: Change the praat template: 
# Set the Mount point (this will need to be run locally)
newls = []
numbered_wav_fp = str(numbered_wavdir) +'/'
with open('./preproc_func/formants/praat_template.txt', 'r') as f: 
    l = f.readlines()
    newls = [ll.replace('{filepath}', numbered_wav_fp) for ll in l]
    newls = [ll.replace('{mount_point}', mount_point) for ll in newls]
    
with open(f'./preproc_func/formants/praat_scripts/praat_{subject}_{block}.txt', 'w') as f: 
    f.writelines(newls)

#### Instructions for formant extraction from here: 
1. Open Praat
2. On mac, click on Praat, and hit Open Praat Script
3. Load up the script
4. Hit run script. 

In [ ]:
formant_fp = preproc_root /  subject_block / 'formants'
!rm -rf $formant_fp
formant_fp.mkdir(parents=True, exist_ok=True)

In [ ]:
from preproc_func.formants.postprocess_formants import postprocess_formants

In [ ]:
from preproc_func.file_org import rename_files
postprocess_formants(numbered_wavdir, formant_fp)
rename_files(str(formant_fp), ind_to_time)